# Control Plane: Connect MS Agent Framework Agents using AzureAIClient
----
This notebook focuses on the **Control Plane** perspective in Azure AI Foundry and shows how to do lightweight checks with the **MS Azure Framework SDK (1.0.0b251223 preview)**.

You will learn how to:

- Inspect deployed agents (Fleet-style view)
- Review Assets (Connections, Agents)

## Table of contents

- [Control Plane overview](#control-plane-overview)
- [Portal vs SDK](#portal-vs-sdk)
- [Setup](#setup)
- [Case 1: Fleet overview](#case-1-fleet-overview)
- [Case 2: Assets](#case-2-assets)
- [Case 3: Quota](#case-3-quota)
- [Additional resources](#additional-resources)
- [Wrap-up](#wrap-up)

## Control Plane overview

### What is the Control Plane?

The Control Plane is the centralized operational surface in Azure AI Foundry for monitoring and governing AI resources across projects.

```
Control Plane = monitoring + management + security + governance
```

### Key areas



## Portal vs SDK
**Recommended usage:**
- Portal: real-time monitoring, compliance/security checks, quota management
- SDK: automation, inventory checks, CI/CD validations

| Feature | Portal | SDK (Python) |
|-----|--------|-------------|
| Fleet overview | ✅ dashboards | ⚠️ limited listing |
| Assets view | ✅ visual | ✅ programmatic |
| Compliance dashboards | ✅ real-time | ❌ not supported |
| Quota management | ✅ manage in Portal | ❌ often read-only/limited |
| Admin / RBAC | ✅ manage access | ❌ not supported |

## MS Foundry Control Plane Accessibility Across Agent Platforms

| Feature        | Foundry Agent | MS Agent Framework | Hosted Agent |
|---------------|---------------|--------------------|--------------|
| Fleet overview | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |
| Asset view    | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |

**MAF onboard to MS Foundry Control Plane:**
- MAF + Foundry Agent Wrapper (AzureAIClient)
- MAF → LangGraph → Container → Hosted Agent

**Hosted Agent limitation (2026-01-07):**
- **Region availability:** Hosted Agents are currently available only in **North Central US**.
- **Cross-region deployment:** If you try to deploy Hosted Agents in another region, you may see: *“Hosted Agents are not enabled in this region”* (**HTTP 400 BadRequest**).
- **Network isolation:** Hosted Agents currently can’t be created using the standard setup in **network-isolated** Foundry resources.
- **ACR requirement:** Hosted Agents can be deployed only when **Azure Container Registry (ACR)** is in the **same resource group** and the **same region (North Central US)**.
- **Last verified:** 2026-01-07

## MS Foundry Tools Integration Support Across Agent Platforms 
| Tool | Foundry Agent | MS Agent Framework | Hosted Agent |
|-----|--------|-------------|-------------|
| bing grounding | ✅ Yes | ✅ Yes | ✅ Yes |
| code interpreter | ✅ Yes | ✅ Yes | ✅ Yes |
| browser automation | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| computer use | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| fabric (Microsoft Fabric) | ✅ Yes (Preview) | ✅ Yes | ❌ No |
| file search | ✅ Yes | ✅ Yes (RAG / AI Search) | ❌ No (Not Built-in) |
| custom function | ✅ Yes | ✅ Yes | ✅ Yes |
| image generation | ✅ Yes (Preview) | ✅ Yes (Preview) | ✅ Yes |
| mcp (Model Context Protocol) | ✅ Yes | ✅ Yes | ✅ Yes |


## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [2]:
# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
import json
import os
import subprocess
from dotenv import load_dotenv
from agent_framework.azure import AzureAIClient
from agent_framework import ChatResponse, HostedCodeInterpreterTool
from azure.identity import DefaultAzureCredential

load_dotenv(override=True)

possible_paths = [
    '/opt/homebrew/bin',  # macOS (Apple Silicon)
    '/usr/local/bin',     # macOS (Intel) / Linux
    '/usr/bin',           # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

🔍 Azure CLI found: /anaconda/envs/azureml_py38/bin//az
✅ PATH looks good already

PATH (first 150 chars): /anaconda/envs/azureml_py38/bin/:/afh/code/agent-operator-lab/.venv/bin:/home/azureuser/.vscode-server/cli/servers/Stable-994fd12f8d3a5aa16f17d42c041e...


In [3]:
# Load Foundry project settings from .foundry_config.json

import os

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
TENANT_ID = config.get('TENANT_ID')
PROJECT_NAME = config.get('PROJECT_NAME', 'proj-default')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')
os.environ['AZURE_AI_PROJECT_ENDPOINT'] = AZURE_AI_PROJECT_ENDPOINT or ''

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Azure AI Project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")

# Fleet overview - list deployed agents
credential = DefaultAzureCredential()

✅ Loaded settings from '../0_setup/.foundry_config.json'.

📌 Foundry name: foundry-rq90gs
📌 Resource group: foundry-rg
📌 Location: swedencentral
📌 Azure AI Project endpoint: https://foundry-rq90gs.services.ai.azure.com/api/projects/default-project


# Case 1: Basic MAF Agent Registration overview

## Register a MAF Agent via AzureAIClient

In [5]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    print("WARNING: AZURE_AI_MODEL_DEPLOYMENT_NAME is empty.")
    print("- In Foundry Portal > Models + endpoints, check the deployment Name")
    print("  then set: export AZURE_AI_MODEL_DEPLOYMENT_NAME='<deployment-name>'")
else:
    print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME = {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

AZURE_AI_MODEL_DEPLOYMENT_NAME = gpt-4.1


In [16]:
# uuid with YYYYMMDDhhmmss
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")
print(f"uuid_suffix = {uuid_suffix}")

uuid_suffix = 20260106085056


In [ ]:
# Method 1) Create AzureAIClient + generate a simple response (Prompt Agent + Response)

agent_name = f"maf-agent-{uuid_suffix}"
instructions = "You are a helpful assistant that answers general questions."

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    raise ValueError("AZURE_AI_MODEL_DEPLOYMENT_NAME is empty. Set it in the previous cell.")

async with (
    AzureAIClient(credential=credential).create_agent(
        name=agent_name,
        instructions=instructions,
    ) as agent,
 ):
    try:
        
        result = await agent.run("What is the size of South Korea in square miles?")

        print(f"Agent: {result}\n")


    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")

Agent: South Korea has an area of approximately **38,691 square miles** (100,363 square kilometers).



# Case 2: MAF Agent overview with conversations 

In [21]:
# uuid with YYYYMMDDhhmmss
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")
print(f"uuid_suffix = {uuid_suffix}")

uuid_suffix = 20260106090911


In [ ]:
# Method 2) Create a maf agent + generate a simple response (Agent + Tool + Conversations/Responses)
from azure.ai.projects import AIProjectClient

agent_name = f"maf-agent-{uuid_suffix}"  
instructions = "You are a helpful assistant that answers general questions."

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    raise ValueError("AZURE_AI_MODEL_DEPLOYMENT_NAME is empty. Set it in the previous cell.")

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
 ):
    agent = None
    conversation = None
    try:
        prompt = "\n".join(
            [
                "Use the code interpreter tool to run Python and compute:",
                "",
                "1) Factorial of 100",
                "2) The first 15 Fibonacci numbers (starting from 0, 1)",
                "3) All prime numbers less than 100",
                "",
                "Return only the results in this format:",
                "- factorial_100: <integer>",
                "- fibonacci_15: [..15 ints..]",
                "- primes_lt_100: [..]",
            ]
        )
        
        conversation = openai_client.conversations.create()
        conversation_id = conversation.id
        print(f"Conversation ID: {conversation_id}")
        
        if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
            raise ValueError("AZURE_AI_MODEL_DEPLOYMENT_NAME is empty. Set it in the previous cell.")

        async with (
            AzureAIClient(credential=credential).create_agent(
                name=agent_name,
                instructions=instructions,
                tools=HostedCodeInterpreterTool(),
            ) as agent,
        ):
            # in order to link the conversation, we create a new thread with the conversation ID
            # (the agent's interactions will be associated with this conversation managed by MS Foundry)
            thread = agent.get_new_thread(service_thread_id=conversation_id)

            print(f"User: {prompt}")
            result = await agent.run(prompt, thread=thread)
            print(f"Agent: {result.text}\n")

    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")


Conversation ID: conv_86b8c69324e45c2000sOCDymjLDDw4xIMHqqvMVQxhbro6LXxx
User: Use the code interpreter tool to run Python and compute:

1) Factorial of 100
2) The first 15 Fibonacci numbers (starting from 0, 1)
3) All prime numbers less than 100

Return only the results in this format:
- factorial_100: <integer>
- fibonacci_15: [..15 ints..]
- primes_lt_100: [..]
Agent: import math

# 1: Factorial of 100
factorial_100 = math.factorial(100)

# 2: First 15 Fibonacci numbers
fibonacci_15 = []
a, b = 0, 1
for _ in range(15):
    fibonacci_15.append(a)
    a, b = b, a + b

# 3: All prime numbers less than 100
def primes_less_than(n):
    primes = []
    for num in range(2, n):
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                break
        else:
            primes.append(num)
    return primes

primes_lt_100 = primes_less_than(100)

factorial_100, fibonacci_15, primes_lt_100 - factorial_100: 93326215443944152681699238856266700490715968264381621468

## Check the tracing of agents


### Steps (MS Foundry portal)
1. Open **Foundry portal**: https://ai.azure.com and navigate to your **project**.
2. In the top navigation, select **Build > Agents > your agent name**.
![Agent Tracing](../images/agent_tracing.png)
3. select **Trace ID** to view traces and spans. 
![Agent Tracing Detail](../images/agent_tracing_detail.png)

## Additional resources

- [Azure AI Foundry Control Plane overview](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/overview?view=foundry)
- [Monitoring across fleet](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/monitoring-across-fleet?view=foundry)
- [Register a custom agent](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/register-custom-agent?view=foundry)
- [Quickstart: create a guardrail policy](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/quickstart-create-guardrail-policy?view=foundry)
- [Manage compliance and security](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-manage-compliance-security?view=foundry)
- [Optimize cost and performance](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-optimize-cost-performance?view=foundry)

# Wrap-up

In this notebook, you did a quick Control Plane-style check using the SDK.

- Good fit for SDK: listing agents/connections, scripting inventory checks
- Portal-first tasks: quota changes, compliance/security dashboards, admin controls

Suggested next steps:
1. Review Fleet/Quota dashboards in the Portal
2. Automate periodic inventory checks in CI
3. Configure governance controls (guardrails/compliance) for production